CCS なら厳しめのパラメタで daligner をやるだけでいけるんじゃないかということでやってみる。

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython.display import display
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.io as pio
pio.templates.default = 'plotly_white'
import logging
import logzero
logzero.loglevel(logging.INFO)

In [78]:
from BITS.util.io import load_pickle, save_pickle
from BITS.util.proc import run_command
from BITS.seq.io import save_fasta
from BITS.plot.plotly import make_line, make_hist, make_scatter, make_layout, show_plot
import numpy as np
import pandas as pd
from logzero import logger
import numpy as np
import pandas as pd
import igraph as ig
from collections import Counter
import plotly.offline as py
import plotly.graph_objs as go

In [11]:
dir_fname = '../work_daligner/'
import os
os.chdir(dir_fname)

## Run daligner

In [146]:
!(rm *.las)

In [147]:
!(HPC.daligner -T16 -k25 -w5 -l3000 -h60 -e.99 -s500 DMEL_CCS_CEN | bash -v)

# Daligner jobs (1)
daligner -k25 -w5 -h60 -e0.99 -l3000 -s500 -T16 DMEL_CCS_CEN DMEL_CCS_CEN && mv DMEL_CCS_CEN.DMEL_CCS_CEN.las DMEL_CCS_CEN.las
# Check initial .las files jobs (1) (optional but recommended)
LAcheck -vS DMEL_CCS_CEN DMEL_CCS_CEN
  DMEL_CCS_CEN: 208,412 all OK


In [62]:
!(LAshow -oc DMEL_CCS_CEN.db DMEL_CCS_CEN.las | head -10)


DMEL_CCS_CEN: 208,412 records

   1    44 c   [ 7,848..13,814> x <12,959.. 6,984]  (12,959 x 13,814 bps,13 trace pts)

         7848 
    A  =======+---------->           dif/(len1+len2) = 37/(5966+5975) =  0.62%
    B         <----------+=======
                           6984



In [67]:
!(LAdump -ocdl DMEL_CCS_CEN.db DMEL_CCS_CEN.las | head -10)

+ P 13446
% P 510
P 1 44 c .
L 13814 12959
C 7848 13814 0 5975
D 37
P 1 287 n .
L 13814 11653
C 0 9406 2240 11653
D 15


revcomp の時の b_read の start/end がややこしいので気をつける (LAshow と LAdump で数字が逆)

## Load overlaps from las

In [49]:
def las_to_ovlps(db_prefix):
    ovlps = []
    for line in run_command(f"LAdump -ocdl {db_prefix}.db {db_prefix}.las").split('\n'):
        if line.startswith('P'):
            _, a_read, b_read, strand, _ = line.strip().split(' ')
            a_read, b_read = map(int, (a_read, b_read))
        elif line.startswith('L'):
            _, a_len, b_len = line.strip().split(' ')
            a_len, b_len = map(int, (a_len, b_len))
        elif line.startswith('C'):
            _, a_start, a_end, b_start, b_end = line.strip().split(' ')
            a_start, a_end, b_start, b_end = map(int, (a_start, a_end, b_start, b_end))
        elif line.startswith('D'):
            _, n_diff = line.strip().split(' ')
            n_diff = int(n_diff)
            p_diff = round(n_diff / ((a_end - a_start + b_end - b_start) / 2) * 100, 2)   # in percentage
            ovlps.append((a_read, b_read, strand, a_start, a_end, a_len, b_start, b_end, b_len, n_diff, p_diff))
    return ovlps

In [50]:
ovlps = las_to_ovlps("DMEL_CCS_CEN")

In [51]:
ovlps

[(1, 44, 'c', 7848, 13814, 13814, 0, 5975, 12959, 37, 0.62),
 (1, 287, 'n', 0, 9406, 13814, 2240, 11653, 11653, 15, 0.16),
 (1, 368, 'c', 10353, 13814, 13814, 0, 3466, 12602, 132, 3.81),
 (1, 514, 'c', 10485, 13814, 13814, 0, 3329, 13424, 11, 0.33),
 (1, 644, 'c', 10100, 13814, 13814, 0, 3719, 12144, 7, 0.19),
 (2, 102, 'n', 0, 4016, 13059, 8120, 12133, 12133, 6, 0.15),
 (2, 339, 'n', 5076, 13059, 13059, 0, 7943, 12610, 96, 1.21),
 (3, 127, 'c', 0, 3494, 12117, 9916, 13414, 13414, 27, 0.77),
 (3, 731, 'c', 8331, 12117, 12117, 0, 3788, 12858, 11, 0.29),
 (4, 220, 'n', 0, 7200, 12867, 5397, 12593, 12593, 18, 0.25),
 (4, 681, 'n', 0, 3676, 12867, 9228, 12897, 12897, 135, 3.68),
 (5, 37, 'n', 0, 6518, 14494, 5242, 11777, 11777, 243, 3.72),
 (5, 37, 'n', 0, 4368, 14494, 7396, 11777, 11777, 143, 3.27),
 (5, 37, 'n', 0, 4009, 14494, 7759, 11777, 11777, 157, 3.91),
 (5, 37, 'n', 0, 3291, 14494, 8474, 11777, 11777, 118, 3.58),
 (5, 37, 'n', 0, 3650, 14494, 8117, 11777, 11777, 125, 3.42),
 (5, 3

## Filter by %diff and remove duplicated overlaps

In [55]:
ovlps = list(filter(lambda ovlp: ovlp[0] < ovlp[1] and ovlp[-1] < 1., ovlps))

In [56]:
ovlps

[(1, 44, 'c', 7848, 13814, 13814, 0, 5975, 12959, 37, 0.62),
 (1, 287, 'n', 0, 9406, 13814, 2240, 11653, 11653, 15, 0.16),
 (1, 514, 'c', 10485, 13814, 13814, 0, 3329, 13424, 11, 0.33),
 (1, 644, 'c', 10100, 13814, 13814, 0, 3719, 12144, 7, 0.19),
 (2, 102, 'n', 0, 4016, 13059, 8120, 12133, 12133, 6, 0.15),
 (3, 127, 'c', 0, 3494, 12117, 9916, 13414, 13414, 27, 0.77),
 (3, 731, 'c', 8331, 12117, 12117, 0, 3788, 12858, 11, 0.29),
 (4, 220, 'n', 0, 7200, 12867, 5397, 12593, 12593, 18, 0.25),
 (5, 150, 'c', 11134, 14494, 14494, 0, 3349, 14370, 16, 0.48),
 (5, 194, 'c', 0, 8080, 14494, 4263, 12335, 12335, 12, 0.15),
 (5, 387, 'c', 0, 4781, 14494, 8532, 13310, 13310, 5, 0.1),
 (5, 477, 'c', 0, 8778, 14494, 4545, 13299, 13299, 40, 0.46),
 (7, 197, 'c', 0, 4629, 12604, 6776, 11411, 11411, 22, 0.47),
 (8, 13, 'c', 0, 9552, 12497, 2341, 11909, 11909, 20, 0.21),
 (8, 160, 'n', 8872, 12497, 12497, 0, 3632, 13348, 34, 0.94),
 (8, 348, 'n', 0, 11214, 12497, 725, 11945, 11945, 20, 0.18),
 (10, 109, 

In [57]:
len(ovlps)

556

## Construct a string graph from the filtered overlaps

In [82]:
def construct_string_graph(overlaps):
    nodes, edges = set(), set()
    for overlap in overlaps:
        f_id, g_id, strand, a_start, a_end, a_len, b_start, b_end, b_len, n_diff, p_diff = overlap
        
        if strand == 'c':
            # a_read[a_start:a_end] ~~ strand(b_read[b_start:b_end]) となるようにする
            # b の座標は常に b_read を forward に見た時のもの。配列の切り出しが先。revcomp は後。
            b_start, b_end = b_len - b_end, b_len - b_start
            
        if a_start > 0:
            if strand == 'n':
                if b_end == b_len:
                    """
                         f.B               f.E
                      f  ----------------->
                      g     --------->
                          g.B        g.E
                    """
                    overlap_type = "contains"
                else:
                    """
                         f.B         f.E
                      f  ----------->
                      g         ------------->
                                g.B           g.E
                    """
                    overlap_type = "suffix-prefix"
            else:
                if b_start == 0:
                    """
                         f.B               f.E
                      f  ----------------->
                      g     <---------
                          g.E        g.B
                    """
                    overlap_type = "contains"
                else:
                    """
                         f.B         f.E
                      f  ----------->
                      g         <-------------
                                g.E           g.B
                    """
                    overlap_type = "suffix-suffix"
        else:
            if a_end == a_len:
                overlap_type = "contained"
            else:
                if strand == 'n':
                    """
                                f.B         f.E
                      f          ----------->
                      g    ----------->
                         g.B          g.E
                    """
                    if b_start == 0:
                        overlap_type = "contains"
                    else:
                        overlap_type = "prefix-suffix"
                else:
                    """
                                f.B         f.E
                      f          ----------->
                      g    <-----------
                         g.E           g.B
                    """
                    if b_end == b_len:
                        overlap_type = "contains"
                    else:
                        overlap_type = "prefix-prefix"

        if overlap_type in ["contains", "contained"]:
            continue
        elif overlap_type == "suffix-prefix":
            nodes.update(["%s:B" % g_id,
                          "%s:B" % f_id,
                          "%s:E" % f_id,
                          "%s:E" % g_id])
            edges.update([("%s:B" % g_id, "%s:B" % f_id, a_start, p_diff),
                          ("%s:E" % f_id, "%s:E" % g_id, b_len - b_end, p_diff)])
        elif overlap_type == "suffix-suffix":
            nodes.update(["%s:E" % g_id,
                          "%s:B" % f_id,
                          "%s:E" % f_id,
                          "%s:B" % g_id])
            edges.update([("%s:E" % g_id, "%s:B" % f_id, a_start, p_diff),
                          ("%s:E" % f_id, "%s:B" % g_id, b_start, p_diff)])
        elif overlap_type == "prefix-suffix":
            nodes.update(["%s:B" % f_id,
                          "%s:B" % g_id,
                          "%s:E" % g_id,
                          "%s:E" % f_id])
            edges.update([("%s:B" % f_id, "%s:B" % g_id, b_start, p_diff),
                          ("%s:E" % g_id, "%s:E" % f_id, a_len - a_end, p_diff)])
        else:   # prefix-prefix
            nodes.update(["%s:B" % f_id,
                          "%s:E" % g_id,
                          "%s:B" % g_id,
                          "%s:E" % f_id])
            edges.update([("%s:B" % f_id, "%s:E" % g_id, b_len - b_end, p_diff),
                          ("%s:B" % g_id, "%s:E" % f_id, a_len - a_end, p_diff)])

    return ig.Graph.DictList(edges=(dict(source=s, target=t, length=l, diff=p) for s, t, l, p in edges),
                             vertices=None,
                             directed=True)

def transitive_reduction(sg):
    v_mark = ["vacant" for v in sg.vs]
    e_reduce = {e.tuple: False for e in sg.es}
    FUZZ = 10   # in bp; this length is in general shorter than unit length, thus we accept no unit shifts

    for v in sg.vs:
        if v.outdegree() == 0:
            continue

        oes = sorted(sg.es.select(_source=v.index), key=lambda x: x["length"])
        longest = oes[-1]["length"] + FUZZ
        for oe in oes:
            v_mark[oe.target] = "inplay"

        for oe in oes:
            if v_mark[oe.target] == "inplay":
                ooes = sorted(sg.es.select(_source=oe.target), key=lambda x: x["length"])
                for ooe in ooes:
                    if oe["length"] + ooe["length"] <= longest and v_mark[ooe.target] == "inplay":
                        v_mark[ooe.target] = "eliminated"

        for oe in oes:
            ooes = sorted(sg.es.select(_source=oe.target), key=lambda x: x["length"])
            if len(ooes) > 1:
                shortest = ooes[0].target
                if v_mark[shortest] == "inplay":
                    v_mark[shortest] == "eliminated"
            for ooe in ooes:
                if ooe["length"] < FUZZ and v_mark[ooe.target] == "inplay":
                    v_mark[ooe.target] = "eliminated"

        for oe in oes:
            if v_mark[oe.target] == "eliminated":
                e_reduce[oe.tuple] = True   # TODO: confirm revcomp edges will be also removed in the same way
            v_mark[oe.target] = "vacant"

    # Re-construct a graph
    return ig.Graph.DictList(edges=(dict(source=e["source"],
                                         target=e["target"],
                                         length=e["length"],
                                         diff=e["diff"])
                                    for e in sg.es
                                    if not e_reduce[e.tuple]),
                             vertices=None,
                             directed=True)

In [83]:
sg = transitive_reduction(construct_string_graph(ovlps))

In [74]:
def draw_graph(sg):
    E = [e.tuple for e in sg.es]
    N = sg.vcount()
    pos = sg.layout('kk')

    edge_trace = go.Scatter(x=[i for l in [(pos[s][0], pos[t][0], None) for s, t in E] for i in l],
                            y=[i for l in [(pos[s][1], pos[t][1], None) for s, t in E] for i in l],
                            line=dict(width=0.5, color='black'),
                            mode='lines')

    shapes = [make_line(pos[s][0] + (pos[t][0] - pos[s][0]) * 0.7,
                        pos[s][1] + (pos[t][1] - pos[s][1]) * 0.7,
                        pos[t][0],
                        pos[t][1],
                        "black",
                        4,
                        "below")
              for s, t in E]

    node_trace = go.Scatter(x=[pos[node][0] for node in range(N)],
                            y=[pos[node][1] for node in range(N)],
                            mode='markers',
                            marker=dict(
                                showscale=False,
                                colorscale='YlGnBu',
                                reversescale=True,
                                size=10,
                                line=dict(width=2)))

    layout = go.Layout(width=1000, height=1000,
                       xaxis=dict(showgrid=False,
                                  zeroline=True,
                                  zerolinecolor="yellow",
                                  showticklabels=True),
                       yaxis=dict(showgrid=False,
                                  zeroline=True,
                                  zerolinecolor="yellow",
                                  showticklabels=True),
                       shapes=shapes,
                       hovermode='closest',
                       margin=go.layout.Margin(l=0, r=0, b=0, t=0),
                       showlegend=False)
    fig = go.Figure(data=[edge_trace, node_trace], layout=layout)
    py.iplot(fig)
    return pos

In [85]:
draw_graph(sg)

<Layout with 1054 vertices and 2 dimensions>

### Look at connected components

In [88]:
cc = [(g, g.vcount()) for g in sg.clusters(mode="weak").subgraphs() if g.vcount() >= 5]

In [89]:
cc

[(<igraph.Graph at 0x2ab857cc0a98>, 7),
 (<igraph.Graph at 0x2ab855d66048>, 6),
 (<igraph.Graph at 0x2ab855d66318>, 5),
 (<igraph.Graph at 0x2ab855d665e8>, 9),
 (<igraph.Graph at 0x2ab855d666d8>, 6),
 (<igraph.Graph at 0x2ab855d667c8>, 8),
 (<igraph.Graph at 0x2ab855d669a8>, 5),
 (<igraph.Graph at 0x2ab855d66a98>, 7),
 (<igraph.Graph at 0x2ab855d66b88>, 5),
 (<igraph.Graph at 0x2ab855d66e58>, 12),
 (<igraph.Graph at 0x2ab855d6b048>, 6),
 (<igraph.Graph at 0x2ab855d6b138>, 8),
 (<igraph.Graph at 0x2ab855d6b4f8>, 6),
 (<igraph.Graph at 0x2ab855d6b6d8>, 6),
 (<igraph.Graph at 0x2ab855d6b7c8>, 5),
 (<igraph.Graph at 0x2ab855d6b8b8>, 7),
 (<igraph.Graph at 0x2ab855d6bd68>, 7),
 (<igraph.Graph at 0x2ab855d71318>, 5),
 (<igraph.Graph at 0x2ab855d714f8>, 5),
 (<igraph.Graph at 0x2ab855d716d8>, 7),
 (<igraph.Graph at 0x2ab855d718b8>, 8),
 (<igraph.Graph at 0x2ab855d719a8>, 14),
 (<igraph.Graph at 0x2ab855d71a98>, 14),
 (<igraph.Graph at 0x2ab855d71b88>, 5),
 (<igraph.Graph at 0x2ab855d75048>, 5

In [90]:
for g, s in cc:
    draw_graph(g)

## A bit looser params for overlap filtering

In [244]:
ovlps = las_to_ovlps("DMEL_CCS_CEN")

In [245]:
ovlps = list(filter(lambda ovlp: ovlp[0] < ovlp[1] and ovlp[-1] < 1.2 and ovlp[4] - ovlp[3] >= 1000, ovlps))

In [246]:
len(ovlps)

590

In [247]:
sg = transitive_reduction(construct_string_graph(ovlps))

In [248]:
cc = [(g, g.vcount()) for g in sg.clusters(mode="weak").subgraphs() if g.vcount() >= 10]

In [249]:
len(cc)

8

In [250]:
cc

[(<igraph.Graph at 0x2ab85769aa98>, 12),
 (<igraph.Graph at 0x2ab85398b9a8>, 12),
 (<igraph.Graph at 0x2ab85398bb88>, 10),
 (<igraph.Graph at 0x2ab853982228>, 16),
 (<igraph.Graph at 0x2ab8538bb8b8>, 16),
 (<igraph.Graph at 0x2ab855d4b408>, 12),
 (<igraph.Graph at 0x2ab854eba318>, 10),
 (<igraph.Graph at 0x2ab854ebae58>, 12)]

In [251]:
for g, s in cc:
    draw_graph(g)

In [252]:
# length of contig(s) in each connected component (just a rough estimate)
# NOTE: be careful also revcomp exists
s = 0
for c, n in cc:
    l = sum([e["length"] for e in c.es])
    s += l
    print(l)
print(s / 2)

59234
50406
51090
88429
122655
65931
37268
50106
262559.5


重複して計上しているものも多数あるが、それでも推定長 ~310 Kbp のほぼすべてをすでにカバーしていることは間違いない。これはつまり全体的にほぼユニーク配列として見なせる (とは言っても 2% diff でダメになる程度だが) ようになっているということで、完全なアセンブリが可能であることを示唆する。

## Conclusion

CLR と比べて格段にユニークな overlap を絞りやすくなって、いくつかの領域については特に難しいことをせずともコンティグにできそうだ。とはいえ、セントロメア全体をまたぐようなコンティグにはあと一歩届かず、より丁寧な overlap が必要であることがわかった。